In [20]:
import os

In [21]:
%pwd

'd:\\kidney'

In [11]:
os.chdir("../")

In [12]:
%pwd


'd:\\kidney'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [23]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [24]:
from pathlib import Path

# CONFIG_FILE_PATH = Path("config/config.yaml").resolve()
# CONFIG_FILE_PATH = Path("params.yaml").resolve()



class ConfigurationManager:
    
   

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "KidneyZIP")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [27]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
         # Modify the model for 4 classes
        if self.model.output_shape[-1] != 4:
            x = self.model.layers[-2].output  # Take output from the second-last layer
            output = tf.keras.layers.Dense(4, activation='softmax')(x)  # Add 4-class output layer
            self.model = tf.keras.Model(inputs=self.model.input, outputs=output)

        # Compile the model for training
            self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adjust learning rate if needed
            loss="categorical_crossentropy",  # Use this loss for multi-class classification
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        
        
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [29]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2024-12-05 15:21:08,637: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-05 15:21:08,641: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-05 15:21:08,641: INFO: common: created directory at: artifacts]
[2024-12-05 15:21:08,657: INFO: common: created directory at: artifacts\training]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
Epoch 1/10
622/622 [==============================] - 3020s 5s/step - loss: 13.9801 - accuracy: 0.4084 - val_loss: 13.3107 - val_accuracy: 0.3056
Epoch 2/10
622/622 [==============================] - 2505s 4s/step - loss: 7.4484 - accuracy: 0.5355 - val_loss: 18.2706 - val_accuracy: 0.2335
Epoch 3/10
622/622 [==============================] - 2626s 4s/step - loss: 5.3662 - accuracy: 0.6057 - val_loss: 4.0300 - val_accuracy: 0.6004
Epoch 4/10
622/622 [==============================] - 1949s 3s/step - loss: 4.7997 - accuracy: 0.6321 - val_loss: 5.8421 - val_accuracy: 0.6718
Epoch 5